In [6]:
## Biermann Deck Maker
## (c) F. S. Tsung 2019

##
## library and constants are here
## What I do, and what I recommend that you do as well is to use Github to load my library, and include 
## it in the path as shown below.  
##
## the constants are in CGS units, I suggest that you do the same.  The notebook assumes CGS units.
##

import sys
# on my mac pro
# sys.path.append('/Users/tsung/Documents/Git/python-tsung') # go to parent dir
# on my laptop
sys.path.append('/Users/FrankTsung/vst-extra/FranksFolder/Work/codes/forks/python-tsung')


# import hfhi
import numpy as np
import hfhi

me = 9.1095e-28
qe = 4.8032e-10
c_cgs = 2.998e10

mypi = 3.1415926

#### 2D Biermann Deck Maker



In [8]:
## first some common constants 




In [3]:
## enter the basic
##



## in the simulations, we adopt the normalization that the density is normalized to n0, speed is normalized to c, and q/m for 
## electrons is -1.  Therefore, under these conditions, 
## nppc -> number of particles per cell

## In our normalizations, q/m for electrons is -1
## n0 is 1
## so electron plasma frequency is 1 by definition
## and speed of light is 1
## so length 

nppc = 64

n0 = 1.0

nb = 0.1

vth0 = 0.2

vb = 0.01

Ln = 20

Lt = 40


length = 15/8 * Lt * 2

print('System Length =',repr(length))

duration = 1300

print('Total Duration = ', repr(duration),'\omega_{pe}')


System Length = 150.0
Total Duration =  1300 \omega_{pe}


In [9]:

# let n0 = 1, so omega_pe = 1.

omega_pe = np.sqrt(n0)
debye_length= vth0 / omega_pe 

# print(debye_length)


resolution = 0.9 * debye_length

# for 2D it's sqrt(1/2), for 3D the stepsize is sqrt(1/3)
# due to the CFL condition of the FDTD scheme.
#
#
timestep = 0.98 * np.sqrt(0.5) * resolution

total_cells = int(length/resolution*length/resolution)

total_steps = duration / timestep

num_par = total_cells * nppc

total_cpus = 4

# ------------------------------------------------------------------
# --  do not touch the lines below, the magic is happening below ---
# ------------------------------------------------------------------



num_par_max = 1.25 * num_par/total_cpus


print('Length in normalized units is : ' +repr(length))
print('Grid size in normalized unit is : '+repr( resolution ))
print('total number of cells = '+repr(total_cells))
print('total number of particles = '+repr(total_cells*nppc/1e6)+' millions')
print('total number of timesteps = '+repr(total_steps))
print('CPU estimate = '+repr(total_steps*total_cells*nppc*2e-7/3600)+' hours')
print('num_par_max estimate = '+repr(num_par_max))

Length in normalized units is : 150.0
Grid size in normalized unit is : 0.18000000000000002
total number of cells = 694444
total number of particles = 44.444416 millions
total number of timesteps = 10422.208792998999
CPU estimate = 25.733832401939189 hours
num_par_max estimate = 13888880.0


In [5]:
## Here we make the namelist for electrons

## electron namelist


print('species ')
print(' {')
# print(' num_par_max = '+repr(int(num_par_max))+' ,')
print(' rqm = -1.0 , ')
print(' num_par_x(1,2) = 8,8, ')
print(' n_sort = 25, ')
print('}')

print('udist')
print('{')

print(' uth(1:3) = '+repr(hfhi.vth(te)/c_cgs)+' , '+repr(hfhi.vth(te)/c_cgs)+' , '+repr(hfhi.vth(te)/c_cgs)+' , ')
print(' ufl(1:3) = 0.0, 0.0, 0.0, ')
print('}')

print('profile')
print('{')
print('profile_type(1,2)= \"math func\", \"math func\"' )
print('math_func_expr = \" (',repr(n0),'-',repr(nb),') * cos( 3.1415926 * sqrt())'    '"')
print(' density = '+ repr(density)+' ,')
print('}')

print('spe_bound')
print('{')

print(' type(1:2,1) = "thermal", thermal", ')
print(' uth_bnd(1:3,1,1) = '+repr(hfhi.vth(te)/c_cgs)+' , '+repr(hfhi.vth(te)/c_cgs)+' , '+repr(hfhi.vth(te)/c_cgs)+' , ')
print(' uth_bnd(1:3,2,1) = '+repr(hfhi.vth(te)/c_cgs)+' , '+repr(hfhi.vth(te)/c_cgs)+' , '+repr(hfhi.vth(te)/c_cgs)+' , ')
# print(' ufl(1:3) = 0.0, 0.0, 0.0, ')
print(' thermal_type(1:2,1) = "half max","half max", ')
print('}')


species 
 {
 rqm = -1.0 , 
 num_par_x(1,2) = 8,8, 
 n_sort = 25, 
}
udist
{
profile+type(1,2)= "math func", "math func"
 uth(1:3) = 0.11703176667042624 , 0.11703176667042624 , 0.11703176667042624 , 
 ufl(1:3) = 0.0, 0.0, 0.0, 
}
profile
{
 density = 0.028 ,
}
spe_bound
{
 type(1:2,1) = "thermal", thermal", 
 uth_bnd(1:3,1,1) = 0.11703176667042624 , 0.11703176667042624 , 0.11703176667042624 , 
 uth_bnd(1:3,2,1) = 0.11703176667042624 , 0.11703176667042624 , 0.11703176667042624 , 
 thermal_type(1:2,1) = "half max","half max", 
}


In [12]:
## ion namelist
# A = Mass Number (proton + neutrons)
# Z = Atomic Number
A = 2.5
Z = 1

## ti in eV's
ti = 2500
rqm_i = 1836 * A/Z

vth_i = hfhi.vth(ti)/np.sqrt(1836*A)

print('species ')
print(' {')
print(' num_par_max = '+repr(int(num_par_max))+' ,')
print(' rqm =  '+repr(rqm_i)+' ,')
print(' num_par_x(1) = 256 , ')
print(' n_sort = 25 , ')
print('}')

print('udist')
print('{')
print(' uth(1:3) = '+repr(vth_i/c_cgs)+' , '+repr(vth_i/c_cgs)+' , '+repr(vth_i/c_cgs)+' , ')
print(' ufl(1:3) = 0.0, 0.0, 0.0, ')
print('}')

print('profile')
print('{')
print(' density = '+ repr(density)+' ,')
print('}')

print('spe_bound')
print('{')
print(' type(1:2,1) = "thermal", thermal", ')
print(' uth_bnd(1:3,1,1) = '+repr(vth_i/c_cgs)+' , '+repr(vth_i/c_cgs)+' , '+repr(vth_i/c_cgs)+' , ')
print(' uth_bnd(1:3,2,1) = '+repr(vth_i/c_cgs)+' , '+repr(vth_i/c_cgs)+' , '+repr(vth_i/c_cgs)+' , ')
# print(' ufl(1:3) = 0.0, 0.0, 0.0, ')
print(' thermal_type(1:2,1) = "half max","half max", ')
print('}')


species 
 {
 num_par_max = 228080 ,
 rqm =  4590.0 ,
 num_par_x(1) = 256 , 
 n_sort = 25 , 
}
udist
{
 uth(1:3) = 0.0010323296535963943 , 0.0010323296535963943 , 0.0010323296535963943 , 
 ufl(1:3) = 0.0, 0.0, 0.0, 
}
profile
{
 density = 0.028 ,
}
spe_bound
{
 type(1:2,1) = "thermal", thermal", 
 uth_bnd(1:3,1,1) = 0.0010323296535963943 , 0.0010323296535963943 , 0.0010323296535963943 , 
 uth_bnd(1:3,2,1) = 0.0010323296535963943 , 0.0010323296535963943 , 0.0010323296535963943 , 
 thermal_type(1:2,1) = "half max","half max", 
}


In [73]:
## antenna namelist

print(' antenna{')
print('   vosc = '+repr(hfhi.vosc(i014,lambda_0))+',')
print('   t_rise = 10.0, t_flat=1e6, t_fall=10,')
print('   omega0 = 1.0,')

print(' }')
# antenna
# {
#  a0=0.0065328024757874189,
#  t_rise=100.0,       
#  t_flat=50000.0, omega0=1.0,
# 
#}


 antenna{
   vosc = 0.042317611770142836,
   t_rise = 10.0, t_flat=1e6, t_fall=10,
   omega0 = 1.0,
 }


In [ ]:
! system parameters
! system length  = sqrt(2) * 2 mm ~ 2.8 mm ~ 8000 wavelengths
! Let's try to do a 1/3 of the system, ~ 1mm, with the coldest ion conditions
!
!
!


! density = 3.6 * 10^20 cm^(-3) (4% nc)
! T_e = 8.6 keV (vth = 0.129729c)
! T_i is piecewise constant
!
!
! 2 keV
! 4 keV
! 8 keV
!System length ~ sqrt(2)* 200 microns
! 200 microns (for 526 nm light) is 2389.03
! for a 9 Angstrom shift, the sound speed is roughly 1/560 c
! using Hydrogen mass ratio (sqrt(M/m) ~ 42).
!
node_conf
{

node_number(1) =  432,
if_periodic(1) = .false.,
}

!----------spatial grid----------
grid
{
  nx_p = 253900,
  coordinates = "cartesian",
}

!----------time step and global data dump timestep number----------
time_step
{
  dt     =   0.198,
  ndump  =   20,
}

!----------restart information----------
restart
{
  ndump_fac = 0,
  if_restart=.true.,
}

!----------spatial limits of the simulations----------
!(note that this includes information about
! the motion of the simulation box)
space
{
  xmin(1) =   0.0 ,
  xmax(1) =   50780, 
  if_move= .false.,
}

!----------time limits ----------
time
{
  tmin = 0.0d0, tmax  = 165000.0d0 ,
}

!----------field solver set up----------
el_mag_fld
{
}

!----------boundary conditions for em-fields ----------
emf_bound
{
  type(1:2,1) =  "lindmann", "lindmann",
}

!----------diagnostic for electromagnetic fields----------
diag_emf
{
  ndump_fac = 30,
  reports="e1","e2","e3","b1","b2","b3",
}

!----------number of particle species----------
particles
{  
  num_species =4,
  interpolation='quadratic',
}

!----------information for species 1----------
species
{
  num_par_max = 50000,
  rqm=-1.0,
  num_par_x(1) = 128,
  n_sort=25,
}

udist
{
  uth(1:3) = 0.129729509353,0.129729509353,0.129729509353,
  ufl(1:3) = 0.0d0 , 0.0d0 , 0.0d0,
}

!----------density profile for this species----------
!  number of points in profile along each direction
!
! xmax(1:2) =   807.8385, 913.506,
!  actual profile 269671
profile
{
  density = 0. 04, 
}

!----------boundary conditions for this species----------
spe_bound
{
  type(1:2,1)= "thermal", "thermal",
  uth_bnd(1:3,1,1) = 0.129729509353,0.129729509353,0.129729509353,
  uth_bnd(1:3,2,1) = 0.129729509353,0.129729509353,0.129729509353,
  thermal_type(1:2,1) = "half max","half max",
}

!----------diagnostic for this species----------
diag_species
{
  ndump_fac = 30,
  reports = "charge","ene","q1",
  ndump_fac_pha = 30,
  ndump_fac_raw = 100,
  ps_xmin(1:2) =   0.0 , 0.000d0 ,
  ps_xmax(1:2) =   807.8385, 913.506,
  ps_pmin(1:3) = -0.3 , -0.3, -0.3,
  ps_pmax(1:3) = 1.0, 1.0, 1.0,
  ps_nx(1:2)   =  400,  256,  ps_np(1:3)   =  100,  100,  100,
  if_ps_p_auto(1:3) = .true., .true., .true.,
  phasespaces = "x2x1", "p1x1", "p1p3", "p1p2", "x1p1p2" ,"x2x1_q1",
  ps_gammamin = 1.0, ps_gammamax = 2000.0,   ps_ngamma = 1024,
  if_ps_gamma_auto = .true.,
  raw_gamma_limit=1.2,
  raw_fraction = 0.1,
  n_ene_bins=6,
  ene_bins(1:6)=0.005,0.04,0.08,0.12,0.2,0.5,
  pha_ene_bin='x2x1_q1'
}

!----------information for species 2----------
species
{
  num_par_max = 50000,
  rqm= 4590.375,
  num_par_x(1) = 128,
  n_sort=200,
}
 
udist
{
  uth(1:3) = 0.000842926301919,0.000842926301919,0.000842926301919,
  ufl(1:3) = 0.0d0 , 0.0d0 , 0.0d0,
}
 
!----------density profile for this species----------
!  number of points in profile along each direction
 
!  actual profile 269671
profile
{
  num_x=6,
  fx(1:6,1) = 0.0,   0.04,   0.04,    0.0,    0.0,    0.0,
   x(1:6,1) = 0,     0.00001,  17000, 17000.00001,  1241.5457, 1500.0,
  fx(1:4,2) = 1.0, 1.0, 1.0, 1.0,
   x(1:4,2) = 0.0, 0.2, 0.2, 1500.0,
}
 
!----------boundary conditions for this species----------
spe_bound
{
  type(1:2,1)= "thermal", "thermal",
  pth_bnd(1:3,1,1) = 0.000842926301919,0.000842926301919,0.000842926301919,
  pth_bnd(1:3,2,1) = 0.000842926301919,0.000842926301919,0.000842926301919,
  thermal_type(1:2,1) = "half max","half max",
}
 
!----------diagnostic for this species----------
diag_species
{
  ndump_fac = 30,
  reports = "charge","ene","q1",
  ndump_fac_pha = 30,
  ps_xmin(1:2) =   0.0 , 0.000d0 ,
  ps_xmax(1:2) =   807.8385, 913.506,
  ps_pmin(1:3) = -0.1 , -0.1, -0.1,
  ps_pmax(1:3) = 0.1, 0.1, 0.1,
  ps_nx(1:2)   =  400,  256,  ps_np(1:3)   =  100,  100,  100,
  if_ps_p_auto(1:3) = .true., .true., .true.,
  phasespaces = "x2x1", "p1x1", "p1p3", "p1p2", "x1p1p2" ,"x2x1_q1",
  ps_gammamin = 1.0, ps_gammamax = 2000.0,   ps_ngamma = 1024,
  if_ps_gamma_auto = .true.,
  raw_gamma_limit=1.2,
  raw_fraction = 0.1,
}


!---------- information for species 3, 4keV ions ----------
species
{
  num_par_max = 50000,
  rqm= 4590.375,
  num_par_x(1) = 128,
  n_sort=200,
}

udist
{
  uth(1:3) = 0.0011920778,0.0011920778,0.0011920778,
  ufl(1:3) = 0.0d0 , 0.0d0 , 0.0d0,
}

!----------density profile for this species----------
!  number of points in profile along each direction

!  actual profile 269671
profile
{
  num_x=6,
  fx(1:6,1) = 0.0,   0.04,   0.04,    0.0,    0.0,    0.0,
   x(1:6,1) = 17000,     17000.00001,  34000, 34000.00001,  1241.5457, 1500.0,
  fx(1:4,2) = 1.0, 1.0, 1.0, 1.0,
   x(1:4,2) = 0.0, 0.2, 0.2, 1500.0,
}

!----------boundary conditions for this species----------
spe_bound
{
  type(1:2,1)= "thermal", "thermal",
  pth_bnd(1:3,1,1) = 0.0011920778,0.0011920778,0.0011920778,
  pth_bnd(1:3,2,1) = 0.0011920778,0.0011920778,0.0011920778,
  thermal_type(1:2,1) = "half max","half max",
}

!----------diagnostic for this species----------
diag_species
{
  ndump_fac = 30,
  reports = "charge","ene","q1",
  ndump_fac_pha = 30,
  ps_xmin(1:2) =   0.0 , 0.000d0 ,
  ps_xmax(1:2) =   807.8385, 913.506,
  ps_pmin(1:3) = -0.1 , -0.1, -0.1,
  ps_pmax(1:3) = 0.1, 0.1, 0.1,
  ps_nx(1:2)   =  400,  256,  ps_np(1:3)   =  100,  100,  100,
  if_ps_p_auto(1:3) = .true., .true., .true.,
  phasespaces = "x2x1", "p1x1", "p1p3", "p1p2", "x1p1p2" ,"x2x1_q1",
  ps_gammamin = 1.0, ps_gammamax = 2000.0,   ps_ngamma = 1024,
  if_ps_gamma_auto = .true.,
  raw_gamma_limit=1.2,
  raw_fraction = 0.1,
}
!----------information for species 4, 8keV ions ----------
species
{
  num_par_max = 50000,
  rqm= 4590.375,
  num_par_x(1) = 128,
  n_sort=200,
}

udist
{
  uth(1:3) = 0.0016858526,0.0016858526,0.0016858526,
  ufl(1:3) = 0.0d0 , 0.0d0 , 0.0d0,
}

!----------density profile for this species----------
!  number of points in profile along each direction

!  actual profile 269671
profile
{
  num_x=6,
  fx(1:6,1) = 0.0,   0.04,   0.04,    0.0,    0.0,    0.0,
   x(1:6,1) = 34000,     34000.00001,  51000, 51000.00001,  1241.5457, 1500.0,
  fx(1:4,2) = 1.0, 1.0, 1.0, 1.0,
   x(1:4,2) = 0.0, 0.2, 0.2, 1500.0,
}

!----------boundary conditions for this species----------
spe_bound
{
  type(1:2,1)= "thermal", "thermal",
  pth_bnd(1:3,1,1) = 0.0016858526,0.0016858526,0.0016858526,
  pth_bnd(1:3,2,1) = 0.0016858526,0.0016858526,0.0016858526,
  thermal_type(1:2,1) = "half max","half max",
}

!----------diagnostic for this species----------
diag_species
{
  ndump_fac = 30,
  reports = "charge","ene","q1",
  ndump_fac_pha = 30,
  ps_xmin(1:2) =   0.0 , 0.000d0 ,
  ps_xmax(1:2) =   807.8385, 913.506,
  ps_pmin(1:3) = -0.1 , -0.1, -0.1,
  ps_pmax(1:3) = 0.1, 0.1, 0.1,
  ps_nx(1:2)   =  400,  256,  ps_np(1:3)   =  100,  100,  100,
  if_ps_p_auto(1:3) = .true., .true., .true.,
  phasespaces = "x2x1", "p1x1", "p1p3", "p1p2", "x1p1p2" ,"x2x1_q1",
  ps_gammamin = 1.0, ps_gammamax = 2000.0,   ps_ngamma = 1024,
  if_ps_gamma_auto = .true.,
  raw_gamma_limit=1.2,
  raw_fraction = 0.1,
}




current{}

smooth
 {
  type(1) = "custom",
  order(1)=5,
  swfj(1:3,1,1) = 1,2,1,
  swfj(1:3,2,1) = 1,2,1,
  swfj(1:3,3,1) = 1,2,1,
  swfj(1:3,4,1) = 1,2,1,
  swfj(1:3,5,1) = -5,14,-5,

 }

!----------diagnostic for currents----------
diag_current
{
}

antenna_array
{
  n_antenna=1
}

!
! I0 = 1.7 * 10^14 W/cm^2 
!
antenna
{
  a0=0.0065328024757874189,
  t_rise=100.0,       
  t_flat=50000.0, omega0=1.0,
 
}


